In [20]:
import os
import numpy as np
import librosa.display
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import MobileNet, MobileNetV3Small, MobileNetV3Large, VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve

In [ ]:
X = np.load('../SavedFeatures/X_cqcc.npy')
y = np.load('../SavedFeatures/y_cqcc.npy')

pad_height = (3, 0)  
pad_width = (0, 0)   
X = np.pad(X, pad_width=((0, 0), pad_height, pad_width), mode='constant', constant_values=0)
X.shape

(15918, 32, 109)

In [ ]:
X_train, X_val, y_train, y_val= train_test_split(X, y, test_size = 0.2, random_state = 42)

y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

del X, y  
gc.collect()

In [73]:
def model_vgg16(input_shape=(32, 109, 1), num_classes=2):
    base_model = VGG16(weights= None, include_top=False, input_shape=input_shape)
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  
    x = BatchNormalization()(x)

    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)

    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=x)

In [74]:
mvgg16 = model_vgg16(input_shape=(32, 109, 1), num_classes=2)
mvgg16.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 109, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 109, 64)    │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 109, 64)    │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 54, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 54, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 27, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 27, 256)     │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 27, 256)     │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 8, 27, 256)     │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 13, 256)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 4, 13, 512)     │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 4, 13, 512)     │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 4, 13, 512)     │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 2, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 2, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 2, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 2, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 14,892,002 (56.81 MB)

 Trainable params: 14,890,082 (56.80 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [75]:
mvgg16.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
hist = mvgg16.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - accuracy: 0.8105 - loss: 0.4586 - val_accuracy: 0.8269 - val_loss: 0.3936
Epoch 2/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.8824 - loss: 0.3187 - val_accuracy: 0.9105 - val_loss: 0.2515
Epoch 3/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 23s 36ms/step - accuracy: 0.8897 - loss: 0.2940 - val_accuracy: 0.4974 - val_loss: 1.3038
Epoch 4/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 23s 36ms/step - accuracy: 0.9118 - loss: 0.2409 - val_accuracy: 0.7762 - val_loss: 0.4991
Epoch 5/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.9168 - loss: 0.2264 - val_accuracy: 0.5128 - val_loss: 1.3243
Epoch 6/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.9232 - loss: 0.2044 - val_accuracy: 0.7664 - val_loss: 0.5371
Epoch 7/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.9326 - loss: 0.1941 - val_accuracy: 0.9517 - val_loss: 0.1417
Epoch 8/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.9344 - loss: 0.1772 - 

In [78]:
test_loss, test_accuracy = mvgg16.evaluate(X_val, y_val, verbose=0)
y_pred = mvgg16.predict(X_val)
y_pred_classes = y_pred.argmax(axis=1) 
y_true_classes = y_val


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


In [79]:
y_pred_classes = y_pred.argmax(axis=1) 
y_true_classes = y_val.argmax(axis=1) 

In [ ]:
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print("===================VGG16 - MS===================")
print(f"TAccuracy: {test_accuracy:.5f}")
print(f"F1-Score: {f1:.5f}")
eers = []

# Tính EER 
for i in range(y_pred.shape[1]):  # Run through each class
    # True labels for the current class
    y_true_binary = y_val[:, i]
    y_pred_prob = y_pred[:, i]

    
    fpr, tpr, thresholds = roc_curve(y_true_binary, y_pred_prob)
    fnr = 1 - tpr
    
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]
    eers.append(eer)
    print(f"Class {i}: EER = {eer:.5f} at threshold {eer_threshold:.5f}")


mean_eer = np.mean(eers)
print(f"EER: {mean_eer:.5f}")


===================VGG16 - MS===================
TAccuracy: 0.91912
F1-Score: 0.91732
Class 0: EER = 0.03425 at threshold 0.99924
Class 1: EER = 0.03344 at threshold 0.00077
EER: 0.03384


In [81]:
# Save model as weights in H5 format
mvgg16.save_weights("CQCC_mvgg16.weights.h5")

In [ ]:
del f1, y_true_binary, y_pred_prob
del fpr, tpr, thresholds, fnr, eer_threshold, eer
del eers, mean_eer
del mvgg16, X_train, X_test, y_train, y_test  
gc.collect()

13707